## `GameState` namedtuple

A „GameState” egy [namedtuple](https://docs.python.org/3.5/library/collections.html#collections.namedtuple), amely a játék aktuális állapotát jelzi. Arra használják, hogy segítsen ábrázolni azokat a játékokat, amelyek állapotát nem lehet könnyen ábrázolni, vagy olyan játékokhoz, amelyek memóra táblát igényelnek, mint például a Tic-Tac-Toe.

A "játékállapot" meghatározása a következő:

`GameState = namedtuple('GameState', 'to_move, utility, board, moves')'

* `to_move`: Azt jelzi, hogy kié a következő lépés.

* `utility`: A játék állapotának az eredményét tárolja. Az eredmény a tárolása hasznos, mert amikor egy Minimax vagy Alphabeta keresést végzünk, sok rekurzív hívást generálunk, amelyek egészen a végső állapotokig terjednek. Amikor ezek a rekurzív hívások visszamennek az eredeti hívotthoz, számos játékállapothoz kiszámoltuk az eredményt. Ezeket az eredményeket a megfelelő „GameState”-ben tároljuk, hogy elkerüljük, hogy újra kiszámoljuk őket.

* `board`: egy szótár, amely a játéktáblát tárolja.

* `moves`: Az aktuális pozícióból lehetséges legális mozgások listáját tárolja.

In [2]:
from collections import namedtuple
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

## "Játék" osztály

Vessünk egy pillantást a modulunk „Játék” osztályára. Látjuk, hogy vannak függvényei, nevezetesen "actions", "result", "utility", "terminal_test", "to_move" és "display".

Látjuk, hogy ezeket a funkciókat valójában nincsenek kifejtve. Ez az osztály csak egy absztrakt osztály; úgy kell létrehoznunk az osztályt a játékunkhoz, hogy örököljük ezt a „Játék” osztályt, és megvalósítjuk a "Játékban" említett összes metódust.

Most nézzük meg a "Játék" osztályunk összes metódusát. Ezeket a módszereket meg kell valósítania, amikor új osztályokat hozunk létre az adott játékhoz.

* 'actions(self, state)': Adott játékállapot esetén ez a metódus az összes lehetséges lépést generálja az adott állapotból, listaként vagy generátorként. A generátor visszaadása lista helyett azzal az előnnyel jár, hogy helyet takarít meg, és továbbra is használható listaként.


* 'result(self, state, move)': Adott egy játékállapot és egy lépés, ez a metódus azt a játékállapotot adja vissza, amelyet ezen a játékállapoton végzett lépést eredményez.


* 'utility(self, state, player)': Adott egy végső játékállapot és egy játékos. Ez a metódus visszaadja az eredményt ha az adott játékos végső játékállapotában. Ennek a módszernek a megvalósítása során tegyük fel, hogy a játék állapota egy végső játékállapot. A modul logikája olyan, hogy ez a metódus csak végső játékállapotokban lesz meghívva.


* 'terminal_test(self, state)': Adott játékállapot esetén ennek a metódusnak 'True'-t kell visszaadnia, ha ez a játékállapot végső játékállapot, és 'False' értéket egyébként.


* 'to_move(self, state)': Adott játékállapot esetén ez a metódus visszaadja a következő játékost. Ezeket az információkat általában a játék állapotában tárolják, így ez a módszer csak annyit tesz, hogy kivonja ezeket az információkat, és visszaküldi.


* 'display(self, state)': Ez a metódus kiírja/megjeleníti a játék aktuális állapotát.

In [3]:
class Game:
    """A játék osztály ami nagyon hasonló a probléma osztályhoz. A konstruktort a kezdő állapot beállításához az 
    ezt az osztály megvalósító gyermek osztályban lesz implementálva"""

    def actions(self, state):
        """Vissza adja a megengedett mozgások listáját."""
        raise NotImplementedError

    def result(self, state, move):
        """Vissza adja azt az állapotot, amely egy állapotból való elmozdulás eredménye. """
        raise NotImplementedError

    def utility(self, state, player):
        """A végső állapotnak az eredményét adja vissza a játékosnak."""
        raise NotImplementedError

    def terminal_test(self, state):
        """True értéket add vissza, ha ez a játék végső állapota."""
        return not self.actions(state)

    def to_move(self, state):
        """Adja vissza azt a játékost, akinek a lépése ebben az állapotban van."""
        return state.to_move

    def display(self, state):
        """Jelenítse meg az adott állapotot."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """N-személyes játék játszás."""

        # kezdő állapot beállítása
        state = self.initial

        # Addig tart a játék amíg végső állapotba nem lépünk
        while True:
            # Veszünk egy játékost
            for player in players:
                # Adjon egy lépést a választott játékos
                move = player(self, state)
                # Kérjük vissza annak eredményét, hogy hogyan módosul a játék ha játékos lépést elvégezzük
                state = self.result(state, move)                
                if self.terminal_test(state):
                    # Ha a játékos lépésével előáll egy végső állapot akkor azt kiratjuk és
                    # vissza adjuk kinyert
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

In [4]:
class TicTacToe(Game):
    """Egy TicTacToe játék rendelkezik egy táblával. Az első játékos X-el lép míg
    a második játékos O-val. A játék tárolja a játékosok lépéseit."""

    def __init__(self, h=3, v=3, k=3):
        """TicTacToe konstruktor. Feladata a játék kezdő állapotának megadása."""
        self.h = h # sorok száma
        self.v = v # oszlopok száma
        self.k = k # hány egymást követő X-re vagy O-ra van szükség egy sorban, oszlopban vagy átlósan a győzelemhez
        moves = [(x, y) for x in range(1, h + 1) for y in range(1, v + 1)] # A kezdőállapotból elérhető összes lehetséges mozgás
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves) # kezdő állapot beállítása

    def actions(self, state):
        """A lehetséges lépések. Valójában minden olyan mező amit még nem foglaltak el"""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Mert Illegális lépést nem tehetünk!

        # készítünk egy másolatot a táblára
        board = state.board.copy()

        # A táblára beállítjuk az új lépést. Tehát X vagy O
        board[move] = state.to_move

        # Frissítsük a lehetséges lépések listáját úgy, hogy
        # töröljük belőle a megtett lépést
        moves = list(state.moves)
        moves.remove(move)

        # Állítsuk elő az új állapotot
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Visszaadja az értéket a játékosnak; 1 a győzelemért, -1 a vereségért, 0 egyébként."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """Egy állapot akkor végső, ha nyert, vagy nincsenek üres mezők."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        # Kérjük le a táblát. Ami egy "dic" (szótár) objektum
        board = state.board

        # Járjuk be a táblát
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                # Írjuk ki az értékét ha van olyan eleme a szótárnak (táblának) ha nincs
                # akkor írjunk ki egy pontot
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """Ha 'X' nyer ezzel a lépéssel, adjon vissza 1-et; ha 'O' nyer, akkor -1; különben vissza 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or # oszlopok ellenőrzése
                self.k_in_row(board, move, player, (1, 0)) or # sorok ellenőrzése
                self.k_in_row(board, move, player, (1, -1)) or # / - átló ellenőrzése
                self.k_in_row(board, move, player, (1, 1))): # \ - átló ellenőrzése
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Vissza ad egy igaz értéket ha az adott irányba a játékos kigyűjtötte a megfelelő
        mennyiségű elemet."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n a lépések száma a sorban 
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Mert magát a mozgást kétszer számoltuk
        return n >= self.k

In [5]:
ttt = TicTacToe()
ttt.display(ttt.initial)

. . . 
. . . 
. . . 


In [6]:
my_state = GameState(
    to_move = 'X',
    utility = '0',
    board = {(1,1): 'X', (1,2): 'O', (1,3): 'X',
             (2,1): 'O',             (2,3): 'O',
             (3,1): 'X',
            },
    moves = [(2,2), (3,2), (3,3)]
    )

ttt.display(my_state)

X O X 
O . O 
X . . 


In [7]:
import random

def random_player(game, state):
    """A random játékos választ véletleszerűen egyet a lehetséges lépések közül."""
    return random.choice(game.actions(state)) if game.actions(state) else None

random_player(ttt, my_state)

(3, 3)

In [8]:
ttt.play_game(random_player, random_player)

O X X 
. O X 
X O O 


-1

In [9]:
ttt5 = TicTacToe(h=10, v=10, k=5)
ttt5.display(ttt.initial)

. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 


In [10]:
ttt5.play_game(random_player, random_player)

O X . . . X . O . O 
X X O X X X O X X O 
. O O O . O O O O O 
. X O . . . . . . . 
X . O . O . X O . . 
X O X O O X . O O O 
X O X O X X X . X O 
. . . . X . . X O X 
X O O X . O X O O X 
X X . . X . X . X . 


-1